# Extraction of electrical features (eFeatures) from experimental data

____

## Overview

____

In this tutorial we will see how to extract electrical features (eFeatures), such as spike amplitude, firing frequency, etc... from experimental traces. The eFeatures describe the electrical behavior our neuron model should reproduce.

The steps we will follow are:

* Select and visualize the data.

* Electrophysiological features will be extracted from the voltage traces, thanks to the ** Electrophys Feature Extraction Library ** [eFEL](https://github.com/BlueBrain/eFEL).

* We will use experimental current traces to create protocols that we will use to simulate our neuron model.

* In weeks 10 and 11 we will use the **Blue Brain Python Optimisation Library** [BluePyOpt](https://github.com/BlueBrain/BluePyOpt) to create a model template for the [NEURON simulator](https://www.neuron.yale.edu/neuron/). There you'll see how the morphology you've chosen, the eFeatures and the stimuli will be combined in setting up the optimization of your neuron model.

We first import some useful Python modules.

In [4]:
%load_ext autoreload
%autoreload

import numpy, IPython
import json, os

import matplotlib.pyplot as plt
%matplotlib notebook
plt.rcParams['figure.figsize'] = 10, 10

import collections

from json2html import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Electrophysiology data
In this section we will process the electrophysiological data recorded with patch clamp (current clamp) experiments.

For this example we have chosen to use one negative current step and two positive current steps with different durations. 

We store the data in a Python dictionary.

In [5]:
# Define the directory containing the traces
data_dir = 'data/'
!ls data

exp_APWaveform_ch6_1042.dat   exp_FirePattern_ch6_2049.dat  exp_IV_ch6_40.dat
exp_APWaveform_ch6_1043.dat   exp_FirePattern_ch6_3048.dat  exp_IV_ch6_41.dat
exp_APWaveform_ch6_1044.dat   exp_FirePattern_ch6_3049.dat  exp_IV_ch6_42.dat
exp_APWaveform_ch6_1045.dat   exp_FirePattern_ch6_52.dat    exp_IV_ch6_43.dat
exp_APWaveform_ch6_1046.dat   exp_FirePattern_ch6_53.dat    exp_IV_ch6_44.dat
exp_APWaveform_ch6_1047.dat   exp_FirePattern_ch7_1048.dat  exp_IV_ch6_45.dat
exp_APWaveform_ch6_2042.dat   exp_FirePattern_ch7_1049.dat  exp_IV_ch7_1031.dat
exp_APWaveform_ch6_2043.dat   exp_FirePattern_ch7_2048.dat  exp_IV_ch7_1032.dat
exp_APWaveform_ch6_2044.dat   exp_FirePattern_ch7_2049.dat  exp_IV_ch7_1033.dat
exp_APWaveform_ch6_2045.dat   exp_FirePattern_ch7_3048.dat  exp_IV_ch7_1034.dat
exp_APWaveform_ch6_2046.dat   exp_FirePattern_ch7_3049.dat  exp_IV_ch7_1035.dat
exp_APWaveform_ch6_2047.dat   exp_FirePattern_ch7_52.dat    exp_IV_ch7_1036.dat
exp_APWaveform_ch6_3042.dat   exp_FirePattern_ch7_53

___
### Traces description

* All the recordings you see above represent different **stimuli** (e.g. "APWaveform", "FirePattern", "IV"). 
* Each stimulus comprises different **sweeps** (e.g. "APWaveform*46-51"), of increasing/decreasing amplitudes.
* Each stimulus is repeated multiple times (e.g. APWaveform 46-51, 1042-1047, 2042-2047, 3042-3047 ). In the example above we have four **repetitions** of each stimulus.

Any individual recording has a trace number (e.g. "_1046"). Note that we have pairs of recordings with the same trace number (e.g. "exp_APWaveform_ch7_51.dat" and "exp_APWaveform_ch6_51.dat"). One of them contains the current stimulus (in this case "*ch7*") and the other the voltage response (in this case "*ch6*").
___

We can for example select a sweep from a stimulus (in this case the one with the highest amplitude) and some repetitions.

With the code below we can for example select traces based on trace number and store them in Python dictionaries.

In [6]:
selected_traces = [1047, 2047, 3047, 1049, 2049, 3049, 1041, 2041, 3041]

# Store voltage data in a dictionary step_name : [list of repetitions]
steps_v_dict = collections.OrderedDict({'LongStepNeg': [], 'ShortStepPos': [], 'LongStepPos': []})

# Store current data in a dictionary step_name : [list of repetitions]
steps_i_dict = collections.OrderedDict({'LongStepNeg': [], 'ShortStepPos': [], 'LongStepPos': []})

# Import the glob Python module to interact with the data directory
import glob

files_list = glob.glob1(data_dir, "*.dat")

for file_name in files_list:
    # Get channel and trace number from the file_name
    channel = int(file_name[:-4].split('_')[2][2:])
    tracenum = int(file_name[:-4].split('_')[-1])
    
    # Even channel numbers are voltage traces in this case
    if channel % 2 == 0:
        if "APWaveform" in file_name and tracenum in selected_traces:
            steps_v_dict['ShortStepPos'].append(numpy.fromfile(os.path.join(data_dir,file_name)))
        if "FirePattern" in file_name and tracenum in selected_traces:
            steps_v_dict['LongStepPos'].append(numpy.fromfile(os.path.join(data_dir,file_name)))
        if "IV" in file_name and tracenum in selected_traces:
            steps_v_dict['LongStepNeg'].append(numpy.fromfile(os.path.join(data_dir,file_name)))
            
    # Odd channel numbers are voltage traces in this case        
    elif channel % 2 == 1:
        if "APWaveform" in file_name and tracenum in selected_traces:
            steps_i_dict['ShortStepPos'].append(numpy.fromfile(os.path.join(data_dir,file_name)))
        if "FirePattern" in file_name and tracenum in selected_traces:
            steps_i_dict['LongStepPos'].append(numpy.fromfile(os.path.join(data_dir,file_name)))
        if "IV" in file_name and tracenum in selected_traces:
            steps_i_dict['LongStepNeg'].append(numpy.fromfile(os.path.join(data_dir,file_name)))
        

We can now plot these traces.

In [7]:
# Initialize a figure
fig1, axes = plt.subplots(len(steps_v_dict), sharey = True)

# Plot the voltage traces
for idx, step_name in enumerate(steps_v_dict.keys()):
    for rep, trace in enumerate(steps_v_dict[step_name]):
        data = trace.reshape(len(trace)/2,2)
        axes[idx].plot(data[:,0],data[:,1], label = 'Rep. ' + str(rep+1))
        axes[idx].set_ylabel('Voltage (mV)')
        axes[idx].legend(loc = 'best')
        axes[idx].set_title(step_name)
    axes[-1].set_xlabel('Time (ms)')

<IPython.core.display.Javascript object>

# 2. Electrophysiological features
To build a detailed neuron model, we need to quantify the electrical behavior we want to reproduce. The metrics we use are the eFeatures, that measure parameters describing for instance the shape of the action potential or the firing properties of a neuron (see [here](http://bluebrain.github.io/eFEL/eFeatures.html) for eFeatures description).

In this particular example, we extract distinct features from the three types of voltage traces.
The eFeatures extracted from the data and later from the model will be used to evaluate the results of the simulations. The mean features values, along with the standard deviations will be stored in a .json file.

In [8]:
# Extract features
import efel

# Define stimulus start and end times
steps_info = {'LongStepNeg': [250, 3250], 'ShortStepPos': [250, 475], 'LongStepPos': [250, 3850]} #give begin and end time of the stimulus 

# Prepare the traces for eFEL
def get_features(data):
    # All the traces converted in eFEL format
    efel_traces = {'LongStepNeg': [], 'ShortStepPos': [], 'LongStepPos': []}
    for step_name, step_traces in data.items():
        for rep in step_traces:            
            data = rep.reshape(len(rep)/2,2)
            # A single eFEL trace 
            trace = {}
            trace['T'] = data[:,0]
            trace['V'] = data[:,1] 
            trace['stim_start'] = [steps_info[step_name][0]]
            trace['stim_end'] = [steps_info[step_name][1]]
            trace['name'] = step_name
            
            efel_traces[step_name].append(trace)
    
    features_values = collections.defaultdict(dict)       
    
    features_values['LongStepNeg'] = efel.getMeanFeatureValues(efel_traces['LongStepNeg'], 
                                                                ['time_constant', 'voltage_deflection_begin', 
                                                                'voltage_deflection'])
    
    features_values['LongStepPos'] = efel.getMeanFeatureValues(efel_traces['LongStepPos'], 
                                                               ['mean_frequency', 'adaptation_index2', 
                                                                'ISI_CV', 'doublet_ISI'])
    
    features_values['ShortStepPos'] = efel.getMeanFeatureValues(efel_traces['ShortStepPos'], 
                                                                ['time_to_first_spike', 'AHP_depth', 
                                                                'AP_width', 'AP_height'])    

    return features_values

We can now visualise the feature values we computed, each row in the table corresponds to a repetition of the same step.

In [9]:
efel_features = dict(get_features(steps_v_dict)) # give a dictionnary with the 3 stimuli 
IPython.display.HTML(json2html.convert(json=efel_features))

We compute features mean and standard deviation.

In [10]:
features_dict = collections.OrderedDict()
for step_name, reps in efel_features.items():
    feature_values = collections.defaultdict(list)
    for rep in reps: 
        for feature_name, value in rep.iteritems():
            feature_values[feature_name].append(value)
   
    features_dict[step_name] = {"soma":{}}
    for name, values in feature_values.items():
        features_dict[step_name]["soma"][name] = [numpy.mean(values), numpy.std(values)]
        
IPython.display.HTML(json2html.convert(json=dict(features_dict)))

We write the eFeatures in a json file that we will use later in the exercise.

In [11]:
with open('features.json', 'w') as fp:
    json.dump(features_dict, fp, indent = 4)

___
### Exercise 1 - Plot some spike eFeatures 

Complete the code below in order to extract the 1. action potentials (AP) peak times, 2. their height (maximum overshoot voltage) and 3. the minimum voltage between the spikes of the "LongStepPos" traces and plot them.

The result should be similar to the last figures in this [eFEL example](https://github.com/BlueBrain/eFEL/blob/master/examples/nmc-portal/L5TTPC2.ipynb).
___

In [16]:
# Retrive the LongStepPos traces from the steps_dict dictionary
traces_data = steps_v_dict['LongStepPos']

traces_efel = []

for idx in range(len(traces_data)):
    trace = {}
    trace_data = traces_data[idx]
    trace_data = trace_data.reshape(len(trace_data)/2,2)
    trace['T'] = trace_data[:,0]
    trace['V'] = trace_data[:,1]
    trace['stim_start'] = [steps_info['LongStepPos'][0]]
    trace['stim_end'] = [steps_info['LongStepPos'][1]]
    
    traces_efel.append(trace)
    
print traces_efel

[{'stim_start': [250], 'stim_end': [3850], 'T': array([  0.00000000e+00,   1.00000000e-01,   2.00000000e-01, ...,
         4.09960000e+03,   4.09970000e+03,   4.09980000e+03]), 'V': array([-63.88750076, -63.88750076, -63.88750076, ..., -71.18125153,
       -71.19999695, -71.19374847])}, {'stim_start': [250], 'stim_end': [3850], 'T': array([  0.00000000e+00,   1.00000000e-01,   2.00000000e-01, ...,
         4.09960000e+03,   4.09970000e+03,   4.09980000e+03]), 'V': array([-64.08125305, -64.0562439 , -64.04374695, ..., -71.28125   ,
       -71.27500153, -71.27500153])}, {'stim_start': [250], 'stim_end': [3850], 'T': array([  0.00000000e+00,   1.00000000e-01,   2.00000000e-01, ...,
         4.09960000e+03,   4.09970000e+03,   4.09980000e+03]), 'V': array([-64.19999695, -64.19999695, -64.19999695, ..., -69.96875   ,
       -69.96875   , -70.        ])}]


In [17]:
# Use the efel "getFeatureNames" function to find the names
# for the peak times, action potential height and the the AHP absolute depth

print efel.getFeatureNames()
#'AHP_depth_abs' 'peak_time' 'AP_height'
names = ['peak_time' 'AP_height' 'AHP_depth_abs']

['AHP1_depth_from_peak', 'AHP2_depth_from_peak', 'AHP_depth', 'AHP_depth_abs', 'AHP_depth_abs_slow', 'AHP_depth_diff', 'AHP_depth_from_peak', 'AHP_slow_time', 'AHP_time_from_peak', 'AP1_amp', 'AP1_begin_voltage', 'AP1_begin_width', 'AP1_peak', 'AP1_width', 'AP2_AP1_begin_width_diff', 'AP2_AP1_diff', 'AP2_AP1_peak_diff', 'AP2_amp', 'AP2_begin_voltage', 'AP2_begin_width', 'AP2_peak', 'AP2_width', 'AP_amplitude', 'AP_amplitude_change', 'AP_amplitude_diff', 'AP_amplitude_from_voltagebase', 'AP_begin_indices', 'AP_begin_time', 'AP_begin_voltage', 'AP_begin_width', 'AP_duration', 'AP_duration_change', 'AP_duration_half_width', 'AP_duration_half_width_change', 'AP_end_indices', 'AP_fall_indices', 'AP_fall_rate', 'AP_fall_rate_change', 'AP_fall_time', 'AP_height', 'AP_phaseslope', 'AP_phaseslope_AIS', 'AP_rise_indices', 'AP_rise_rate', 'AP_rise_rate_change', 'AP_rise_time', 'AP_width', 'APlast_amp', 'BAC_maximum_voltage', 'BAC_width', 'BPAPAmplitudeLoc1', 'BPAPAmplitudeLoc2', 'BPAPHeightLoc1',

In [20]:
efeatures = efel.getFeatureValues(traces_efel, names) # Insert here three feature names
fig1, axes = plt.subplots(len(traces_data))

# These are "list comprehensions", a more compat way for writing for loops in Python
[axes[rep].plot(traces_efel[rep]['T'], traces_efel[rep]['V']) for rep in range(len(traces_data))]

# Take inspiration from the line above to plot spike times (x axis) and spike hights (y axis)
#[___ for rep in range(len(traces_data))]

# Take inspiration from the line above to plot spike times (x axis) and AHP depths (y axis)
#[___ for rep in range(len(traces_data))]

AttributeError: 'list' object has no attribute 'getFeatureValues'

## 3. Write out the stimulation protocols

Now it's time to process the current stimuli that were used to record the voltage responses seen above.

We will estimate the stimuli amplitude from the trace and save them in a file "protocols.json". They will be used later on in the project to stimulate your neuron model.

In [ ]:
# Plot the current traces
# Initialize a figure
fig1, axes = plt.subplots(len(steps_i_dict), sharey = True)

for idx, step_name in enumerate(steps_i_dict.keys()):
    for rep, trace in enumerate(steps_i_dict[step_name]):
        data = trace.reshape(len(trace)/2,2)
        axes[idx].plot(data[:,0],data[:,1], label = 'Rep. ' + str(rep+1))
        axes[idx].set_ylabel('Current (nA)')
        axes[idx].legend(loc = 'best')
        axes[idx].set_title(step_name)
    axes[-1].set_xlabel('Time (ms)')

In [ ]:
protocols_dict = collections.OrderedDict()

# Stimuli start and end time
steps_info = {'LongStepNeg': [250, 3250], 'ShortStepPos': [250, 475], 'LongStepPos': [250, 3850]}

# Stimuli holding current and step current amplitudes in nA
amps_info = collections.defaultdict(list)
for step_name in steps_i_dict.keys():
    
    iholds = []
    isteps = []
    for trace in steps_i_dict[step_name]:
        data = trace.reshape(len(trace)/2,2)
        tot_duration = steps_info[step_name][1]+steps_info[step_name][0]
   
        dt = float(tot_duration)/len(data)
        ihold = numpy.mean(data[:,1][0:int(steps_info[step_name][0]/dt)])

        istep = numpy.mean(data[:,1][int(steps_info[step_name][0]/dt):int(steps_info[step_name][1]/dt)])-ihold
        iholds.append(ihold)
        isteps.append(istep)
       
    amps_info[step_name].append(round(numpy.mean(isteps), 4))
    amps_info[step_name].append(round(numpy.mean(iholds), 4)) 
    
#amps_info  = {'LongStepNeg': [-0.01, 0.05], 'ShortStepPos': [0.18,0.05],'LongStepPos': [0.15 ,0.05]}

for step_name, reps in efel_features.items():   
    protocols_dict[step_name] = {"stimuli":[]}
    protocols_dict[step_name]["stimuli"].append({"delay":steps_info[step_name][0],
                                               "amp":amps_info[step_name][0],
                                               "duration":steps_info[step_name][1]-steps_info[step_name][0],
                                               "totduration":steps_info[step_name][1]+steps_info[step_name][0]})
    protocols_dict[step_name]["stimuli"].append({"delay":0,
                                               "amp":amps_info[step_name][1],
                                               "duration":steps_info[step_name][1]+steps_info[step_name][0],
                                               "totduration":steps_info[step_name][1]+steps_info[step_name][0]})
    
IPython.display.HTML(json2html.convert(json=dict(protocols_dict)))


In [ ]:
# Save the protocols in a .json file
with open('protocols.json', 'w') as fp:
    json.dump(protocols_dict, fp, indent = 4)